In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.7`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .config("spark.sql.autoBroadcastJoinThreshold", 0)
    .master("local[*]")
    .getOrCreate()
}

def sc = spark.sparkContext // Разница между def, val, lazy val

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@5e294e26
defined function sc

In [2]:
val accum = sc.longAccumulator("My Accumulator")

sc.parallelize(Array(1, 2, 3, 4), 4).foreach(x => accum.add(x))

accum.value

foreach at cmd1.sc:3

4 / 4

accum: org.apache.spark.util.LongAccumulator = LongAccumulator(id: 0, name: Some(My Accumulator), value: 10)
res1_2: java.lang.Long = 10L

In [3]:
val crimeFacts = spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("data/crime.csv")

crimeFacts.show

csv at cmd2.sc:5

1 / 1

csv at cmd2.sc:5

4 / 4

show at cmd2.sc:7

1 / 1

+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|           STREET|        Lat|        Long|            Location|
+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|     I182070945|         619|             Larceny|  LARCENY ALL OTHERS|     D14|           808|    null|2018-09-02 13:00:00|2018|    9|     Sunday|  13|  Part One|       LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070943|        1402|           Vandalism|           VANDALISM|     C11|   

crimeFacts: DataFrame = [INCIDENT_NUMBER: string, OFFENSE_CODE: int ... 15 more fields]

In [4]:
import spark.implicits._

crimeFacts.groupBy($"OFFENSE_CODE").count.filter($"OFFENSE_CODE" === 1402).explain(true)
crimeFacts.groupBy($"OFFENSE_CODE").count.filter($"OFFENSE_CODE" === 1402).show

== Parsed Logical Plan ==
'Filter ('OFFENSE_CODE = 1402)
+- Aggregate [OFFENSE_CODE#11], [OFFENSE_CODE#11, count(1) AS count#131L]
   +- Relation[INCIDENT_NUMBER#10,OFFENSE_CODE#11,OFFENSE_CODE_GROUP#12,OFFENSE_DESCRIPTION#13,DISTRICT#14,REPORTING_AREA#15,SHOOTING#16,OCCURRED_ON_DATE#17,YEAR#18,MONTH#19,DAY_OF_WEEK#20,HOUR#21,UCR_PART#22,STREET#23,Lat#24,Long#25,Location#26] csv

== Analyzed Logical Plan ==
OFFENSE_CODE: int, count: bigint
Filter (OFFENSE_CODE#11 = 1402)
+- Aggregate [OFFENSE_CODE#11], [OFFENSE_CODE#11, count(1) AS count#131L]
   +- Relation[INCIDENT_NUMBER#10,OFFENSE_CODE#11,OFFENSE_CODE_GROUP#12,OFFENSE_DESCRIPTION#13,DISTRICT#14,REPORTING_AREA#15,SHOOTING#16,OCCURRED_ON_DATE#17,YEAR#18,MONTH#19,DAY_OF_WEEK#20,HOUR#21,UCR_PART#22,STREET#23,Lat#24,Long#25,Location#26] csv

== Optimized Logical Plan ==
Aggregate [OFFENSE_CODE#11], [OFFENSE_CODE#11, count(1) AS count#131L]
+- Project [OFFENSE_CODE#11]
   +- Filter (isnotnull(OFFENSE_CODE#11) && (OFFENSE_CODE#11 = 1402))

show at cmd3.sc:4

4 / 4

show at cmd3.sc:4

1 / 1

...

100 / 100

show at cmd3.sc:4

75 / 75

+------------+-----+
|OFFENSE_CODE|count|
+------------+-----+
|        1402|15154|
+------------+-----+



import spark.implicits._



In [5]:
val offenseCodes = spark.read.option("header", "true").option("inferSchema", "true").csv("data/offense_codes.csv")
offenseCodes.show(100, false)

csv at cmd4.sc:1

1 / 1

csv at cmd4.sc:1

1 / 1

show at cmd4.sc:2

1 / 1

+----+----------------------------------------------------------+
|CODE|NAME                                                      |
+----+----------------------------------------------------------+
|612 |LARCENY PURSE SNATCH - NO FORCE                           |
|613 |LARCENY SHOPLIFTING                                       |
|615 |LARCENY THEFT OF MV PARTS & ACCESSORIES                   |
|1731|INCEST                                                    |
|3111|LICENSE PREMISE VIOLATION                                 |
|2646|LIQUOR - DRINKING IN PUBLIC                               |
|2204|LIQUOR LAW VIOLATION                                      |
|3810|M/V ACCIDENT - INVOLVING �BICYCLE - INJURY                |
|3801|M/V ACCIDENT - OTHER                                      |
|3807|M/V ACCIDENT - OTHER CITY VEHICLE                         |
|3803|M/V ACCIDENT - PERSONAL INJURY                            |
|3805|M/V ACCIDENT - POLICE VEHICLE                             |
|3802|M/V 

offenseCodes: DataFrame = [CODE: int, NAME: string]

In [7]:
import spark.implicits._

val robberyStats = crimeFacts
    .join(offenseCodes, $"CODE" === $"OFFENSE_CODE")
    .filter($"NAME".startsWith("ROBBERY"))
    .groupBy($"NAME")
    .count()
    .orderBy($"count".desc)
    
// robberyStats.show(false)
robberyStats.explain(true)

== Parsed Logical Plan ==
'Sort ['count DESC NULLS LAST], true
+- Aggregate [NAME#185], [NAME#185, count(1) AS count#366L]
   +- Filter StartsWith(NAME#185, ROBBERY)
      +- Join Inner, (CODE#184 = OFFENSE_CODE#11)
         :- Relation[INCIDENT_NUMBER#10,OFFENSE_CODE#11,OFFENSE_CODE_GROUP#12,OFFENSE_DESCRIPTION#13,DISTRICT#14,REPORTING_AREA#15,SHOOTING#16,OCCURRED_ON_DATE#17,YEAR#18,MONTH#19,DAY_OF_WEEK#20,HOUR#21,UCR_PART#22,STREET#23,Lat#24,Long#25,Location#26] csv
         +- Relation[CODE#184,NAME#185] csv

== Analyzed Logical Plan ==
NAME: string, count: bigint
Sort [count#366L DESC NULLS LAST], true
+- Aggregate [NAME#185], [NAME#185, count(1) AS count#366L]
   +- Filter StartsWith(NAME#185, ROBBERY)
      +- Join Inner, (CODE#184 = OFFENSE_CODE#11)
         :- Relation[INCIDENT_NUMBER#10,OFFENSE_CODE#11,OFFENSE_CODE_GROUP#12,OFFENSE_DESCRIPTION#13,DISTRICT#14,REPORTING_AREA#15,SHOOTING#16,OCCURRED_ON_DATE#17,YEAR#18,MONTH#19,DAY_OF_WEEK#20,HOUR#21,UCR_PART#22,STREET#23,Lat#24,L

import spark.implicits._


robberyStats: Dataset[Row] = [NAME: string, count: bigint]

In [8]:
import org.apache.spark.sql.functions.broadcast

val offenseCodesBroadcast = broadcast(offenseCodes)

val robberyStatsWithBroadcast = crimeFacts
    .join(offenseCodesBroadcast, $"CODE" === $"OFFENSE_CODE")
    .filter($"NAME".startsWith("ROBBERY"))
    .groupBy($"NAME")
    .count()
    .orderBy($"count".desc)
    
robberyStatsWithBroadcast.show

run at ThreadPoolExecutor.java:1149

1 / 1

show at cmd7.sc:12

4 / 4

show at cmd7.sc:12

200 / 200

+--------------------+-----+
|                NAME|count|
+--------------------+-----+
|    ROBBERY - STREET| 3056|
|ROBBERY - FIREARM...| 3056|
|     ROBBERY - OTHER|  635|
|ROBBERY ATTEMPT -...|  635|
|ROBBERY - KNIFE -...|  554|
|ROBBERY - COMMERCIAL|  554|
|      ROBBERY - BANK|  181|
|ROBBERY ATTEMPT -...|  181|
|ROBBERY - HOME IN...|  104|
|ROBBERY - CAR JAC...|   86|
|ROBBERY - UNARMED...|    3|
|ROBBERY - KNIFE -...|    1|
|ROBBERY ATTEMPT -...|    1|
|ROBBERY - UNARMED...|    1|
|ROBBERY - UNARMED...|    1|
|ROBBERY - UNARMED...|    1|
+--------------------+-----+



import org.apache.spark.sql.functions.broadcast


offenseCodesBroadcast: Dataset[Row] = [CODE: int, NAME: string]
robberyStatsWithBroadcast: Dataset[Row] = [NAME: string, count: bigint]

In [9]:
import spark.implicits._

val codesTotal = offenseCodes
    .filter('NAME.startsWith("ROBBERY"))
    .select('CODE)
    .distinct
    .count()

val bf = offenseCodes
    .filter('NAME.startsWith("ROBBERY"))
    .stat.bloomFilter('CODE, codesTotal, 0.01)

import org.apache.spark.sql.functions.udf

def mc(s: Int): Boolean = {
    bf.mightContain(s)
}

val mightContain = udf(mc _)

// offenseCodes.filter('NAME.startsWith("ROBBERY")).show(1000)
// offenseCodes.filter(mightContain('CODE)).show(1000)

val robberyStatsWithBf = crimeFacts
    .filter(mightContain('OFFENSE_CODE))
    .join(offenseCodes, 'CODE === 'OFFENSE_CODE)
    .filter('NAME.startsWith("ROBBERY"))
    .groupBy('NAME)
    .count()
    .orderBy('count.desc)

// robberyStatsWithBf.explain(true)
robberyStatsWithBf.show(true)

count at cmd8.sc:5

1 / 1

count at cmd8.sc:5

200 / 200

count at cmd8.sc:5

...

200 / 200

show at cmd8.sc:33

200 / 200

+--------------------+-----+
|                NAME|count|
+--------------------+-----+
|ROBBERY - FIREARM...| 3056|
|    ROBBERY - STREET| 3056|
|ROBBERY ATTEMPT -...|  635|
|     ROBBERY - OTHER|  635|
|ROBBERY - COMMERCIAL|  554|
|ROBBERY - KNIFE -...|  554|
|ROBBERY ATTEMPT -...|  181|
|      ROBBERY - BANK|  181|
|ROBBERY - HOME IN...|  104|
|ROBBERY - CAR JAC...|   86|
|ROBBERY - UNARMED...|    3|
|ROBBERY - KNIFE -...|    1|
|ROBBERY - UNARMED...|    1|
|ROBBERY ATTEMPT -...|    1|
|ROBBERY - UNARMED...|    1|
|ROBBERY - UNARMED...|    1|
+--------------------+-----+



import spark.implicits._


codesTotal: Long = 58L
bf: org.apache.spark.util.sketch.BloomFilter = org.apache.spark.util.sketch.BloomFilterImpl@bd441056
import org.apache.spark.sql.functions.udf


defined function mc
mightContain: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd8$Helper$$Lambda$5725/2013898750@18cca4a4,
  BooleanType,
  Some(List(IntegerType))
)
robberyStatsWithBf: Dataset[Row] = [NAME: string, count: bigint]

In [ ]:
val cachedCrimeFacts = crimeFacts.select("INCIDENT_NUMBER").cache()

cachedCrimeFacts.count()
cachedCrimeFacts.count()

cachedCrimeFacts.unpersist()

In [11]:
class NotSerializableClass(value: Int) {
    def getValue = value
}

val breakIt = new NotSerializableClass(1)

sc.parallelize(Seq(1, 2, 3))
    .map { x =>
        x + breakIt.getValue
    }
    .collect()

collect at cmd10.sc:8

4 / 4

defined class NotSerializableClass
res10_1: Array[Int] = Array(2, 3, 4)

In [ ]:
robberyStats.explain(true)

In [ ]:
crimeFacts.show

In [ ]:
crimeFacts.select($"year", $"month").distinct.count

In [ ]:
class NotSerializable(v: Int) {
    def getV = v
}

val ns = new NotSerializable(1)

sc.parallelize(Array(1, 2, 3)).map { x =>
    // val ns = new NotSerializable(x)
    ns.getV
}.collect()

In [ ]:
crimeFacts.show

In [ ]:
crimeFacts.selectExpr("DISTRICT", "percentile_approx(Lat, 0.5) over (partition by DISTRICT) as medianLat").show

In [ ]:
crimeFacts.createOrReplaceTempView("crimeFacts")
spark.sql("select DISTRICT, percentile_approx(Lat, 0.5) as medianLat from crimeFacts group by DISTRICT").show

In [ ]:
val colList = crimeFacts.columns

val req = s"""select ${colList.mkString(", ")} from target"""

req

In [ ]:
val colsForTarget = List("id", "new_id") ++ colList

crimeFacts.select(colsForTarget: _*)